# Подготовка данных к 5 неделе, часть 1.
Глазунов А.В.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/'My Drive'/Colab_Notebooks/ML_c6_TAXI

/content/gdrive/My Drive/Colab_Notebooks/ML_c6_TAXI


In [0]:
PATH_TO_DATA = 'data'

In [0]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import gc
import statsmodels.api as sm

from itertools import product

In [0]:
def prepare_dataset_geo_time_series(taxi_data,reg_data):

  """  
  Описание

  Данная функция принимает на вход загруженные 
  таблицу с поездками и список зон с координатами

  Возвращает временной ряд частот поездок по времени 
  размера (time_length х regions_length)

  """
  df = taxi_data.copy()
  
  regions = reg_data.copy()

  #границы города, y - широта, x - долгота
  (Ymin,Ymax) = (regions.south.min(),regions.north.max())
  (Xmin,Xmax) = (regions.west.min(), regions.east.max())

  #очистка 
  df.drop(df[df.pickup_latitude <= Ymin].index,inplace =True )
  df.drop(df[df.pickup_latitude >= Ymax].index,inplace =True )
  df.drop(df[df.pickup_longitude <= Xmin].index,inplace =True )
  df.drop(df[df.pickup_longitude >= Xmax].index,inplace =True )
  df.drop(df[df.passenger_count == 0].index,inplace =True)
  df.drop(df[df.trip_distance == 0].index,inplace =True)
  df.drop(df[df.tpep_pickup_datetime == df.tpep_dropoff_datetime].index,inplace =True)

  #столбец времени по часам для построения ряда по часам
  df['time'] = df.tpep_pickup_datetime.apply(lambda x: x[:-6])

  #построение таблицы со строками - часами, столбцами - зонами, 
  #на пересечении - количество поездок в данный час из данной зоны
  binx = sorted(list(regions.west.unique()) + [regions.east.max()])
  biny = sorted(list(regions.south.unique()) + [regions.north.max()])

  hour_counts = []
  time = []
  for (tt,sub_df) in tqdm_notebook(df.groupby('time')):
    x = sub_df.pickup_longitude
    y = sub_df.pickup_latitude

    res = stats.binned_statistic_2d(x,y,x, statistic = 'count', bins = [binx,biny])
    counts = res.statistic.reshape(-1)    

    time.append(pd.to_datetime(tt))
    hour_counts.append(counts)

    
  #датафрейм-результат, отсортированный по времени в формате datetime
  
  time_series = pd.DataFrame(hour_counts,index = time,columns = list(regions.region),dtype=int)
  time_series.sort_index(inplace=True)

  return time_series

In [0]:
%%time
regions = pd.read_csv(os.path.join(PATH_TO_DATA,'regions.csv'),sep=';')

CPU times: user 3.38 ms, sys: 6.44 ms, total: 9.82 ms
Wall time: 377 ms


In [0]:
%%time

months = ['2015-12','2016-01','2016-02','2016-03','2016-04','2016-05','2016-06']
time_series_tables = []

for month in months:
  df = pd.read_csv(os.path.join(PATH_TO_DATA,f'yellow_tripdata_{month}.csv')) 
  time_series_table = prepare_dataset_geo_time_series(df,regions)

  gc.collect()
  
  time_series_tables.append(time_series_table)

  
del df
time_series_7_month = pd.concat(time_series_tables)
time_series_7_month.to_csv(os.path.join(PATH_TO_DATA,'time_series_7_month.csv'))


CPU times: user 7min 24s, sys: 33.5 s, total: 7min 57s
Wall time: 8min 55s
